# 실습 06: 이미지 전처리로 OCR 정확도 향상

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/leecks1119/document_ai_lecture/blob/master/notebooks/Lab06_이미지전처리.ipynb)

## 🎯 학습 목표
- 이미지 품질 평가
- 노이즈 제거, 이진화, 기울기 보정
- 전처리 전후 OCR 성능 비교

## ⏱️ 소요 시간: 40분
## 📊 난이도: ⭐⭐⭐⭐☆


In [ ]:
# 한글 폰트 설치 및 설정 (Colab용)
!apt-get install -y fonts-nanum fonts-nanum-coding fonts-nanum-extra
!fc-cache -fv

# matplotlib 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rc('font', family=fontprop.get_name())
plt.rc('axes', unicode_minus=False)

print("✅ 한글 폰트 설정 완료!")


In [ ]:
# 환경 설정
!pip install -q git+https://github.com/leecks1119/document_ai_lecture.git
!apt-get install -y tesseract-ocr tesseract-ocr-kor


In [ ]:
# 노이즈가 있는 테스트 이미지 생성
from docai_course.preprocessing import DocumentPreprocessor
from PIL import Image, ImageDraw, ImageFont
import cv2
import numpy as np

# 한글 폰트 로드
try:
    font = ImageFont.truetype("C:\\Windows\\Fonts\\malgun.ttf", 20)
except:
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothic.ttf", 20)
    except:
        font = ImageFont.load_default()

# 깨끗한 이미지 생성
img = Image.new('RGB', (600, 300), color='white')
draw = ImageDraw.Draw(img)
text = "계약서\n\n계약일자: 2025년 1월 15일\n계약금액: 1,500,000원\n담당자: 홍길동 (02-1234-5678)"
draw.text((30, 30), text, fill='black', font=font)

# OpenCV 변환 및 노이즈 추가 (더 강하게)
img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
noise = np.random.normal(0, 40, img_cv.shape).astype(np.uint8)
noisy_img = cv2.add(img_cv, noise)

# 기울기 추가 (7도 - 더 명확하게)
rows, cols = noisy_img.shape[:2]
M = cv2.getRotationMatrix2D((cols/2, rows/2), 7, 1)
rotated_img = cv2.warpAffine(noisy_img, M, (cols, rows), borderValue=(255,255,255))

# 추가: 약간의 블러 효과
blurred_img = cv2.GaussianBlur(rotated_img, (3, 3), 0)
cv2.imwrite('noisy_document.jpg', blurred_img)

# 원본도 저장 (비교용)
cv2.imwrite('clean_document.jpg', img_cv)

print("✅ 노이즈 이미지 생성 완료!")


## 🔍 기울기 감지 원리

전처리 과정에서 **자동으로 기울기를 감지하고 보정**합니다:

### 1. 기울기 감지 방법
```python
# 이진화된 이미지에서 텍스트 픽셀 좌표 추출
coords = np.column_stack(np.where(binary > 0))

# 최소 영역 사각형(Minimum Area Rectangle) 계산
angle = cv2.minAreaRect(coords)[-1]
```

### 2. 각도 보정
- **cv2.minAreaRect()**: 모든 텍스트 픽셀을 감싸는 최소 크기 회전 사각형을 찾음
- 이 사각형의 각도 = 텍스트의 기울기
- -45° ~ +45° 범위로 정규화

### 3. 회전 적용
- 감지된 각도만큼 **반대 방향**으로 회전
- 기울기 0.5도 이상일 때만 보정 (미세한 각도는 무시)

### 4. 실제 적용 예시
```
원본 이미지: 7도 기울어짐 감지
→ cv2.minAreaRect 결과: -7도
→ 보정: +7도 회전 → 수평 정렬 완료!
```


In [ ]:
# 기울기 감지 과정 시각화
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 노이즈 이미지 로드
img = cv2.imread('noisy_document.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 이진화
_, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# 텍스트 픽셀 좌표 추출
coords = np.column_stack(np.where(binary < 255))  # 검은색 픽셀 (텍스트)

# 최소 영역 사각형 계산
rect = cv2.minAreaRect(coords)
box = cv2.boxPoints(rect)
box = box.astype(int)  # NumPy 최신 버전 호환

# 감지된 각도
angle = rect[-1]
if angle < -45:
    angle = 90 + angle

# 시각화
img_with_rect = img.copy()
cv2.drawContours(img_with_rect, [box], 0, (0, 255, 0), 3)

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title('원본 이미지 (기울어진 상태)', fontsize=14, fontproperties=fontprop, pad=15)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(img_with_rect, cv2.COLOR_BGR2RGB))
plt.title(f'감지된 기울기: {angle:.2f}도', fontsize=14, fontproperties=fontprop, pad=15, color='red')
plt.axis('off')

# 초록색 박스 설명 추가
plt.text(0.5, -0.05, '초록색 박스 = 최소 영역 사각형 (텍스트를 감싸는 최소 크기)', 
         ha='center', transform=plt.gca().transAxes, fontsize=11, fontproperties=fontprop, color='green')

plt.tight_layout()
plt.show()

print(f"\n📐 감지된 기울기: {angle:.2f}도")
print(f"→ 보정 방향: {-angle:.2f}도 회전 (반대 방향으로 보정)")
print(f"\n✅ 기울기 자동 감지 완료!")


In [ ]:
# 전처리 전후 이미지 비교 시각화
import matplotlib.pyplot as plt
import cv2

# 이미지 로드
clean_img = cv2.imread('clean_document.jpg')
noisy_img = cv2.imread('noisy_document.jpg')
processed_img = cv2.imread('processed_document.jpg')
binary_img = cv2.imread('binary_document.jpg')

# RGB 변환 (matplotlib 표시용)
clean_rgb = cv2.cvtColor(clean_img, cv2.COLOR_BGR2RGB)
noisy_rgb = cv2.cvtColor(noisy_img, cv2.COLOR_BGR2RGB)
processed_rgb = cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB)
binary_rgb = cv2.cvtColor(binary_img, cv2.COLOR_BGR2RGB)

# 4개 이미지 비교 시각화
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

axes[0, 0].imshow(clean_rgb)
axes[0, 0].set_title('① 원본 이미지 (깨끗한 상태)', fontsize=14, fontproperties=fontprop, pad=15)
axes[0, 0].axis('off')

axes[0, 1].imshow(noisy_rgb)
axes[0, 1].set_title('② 노이즈 + 기울기 + 블러', fontsize=14, fontproperties=fontprop, pad=15)
axes[0, 1].axis('off')

axes[1, 0].imshow(processed_rgb)
axes[1, 0].set_title('③ 전처리 후 (노이즈 제거 + 기울기 보정)', fontsize=14, fontproperties=fontprop, pad=15)
axes[1, 0].axis('off')

axes[1, 1].imshow(binary_rgb)
axes[1, 1].set_title('④ 이진화 결과 (OCR 최적화)', fontsize=14, fontproperties=fontprop, pad=15)
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("\n✅ 전처리 과정 시각화 완료!")


In [ ]:
# 적응적 전처리 실행
preprocessor = DocumentPreprocessor(debug_mode=True)

processed_img, binary_img, quality_metrics = preprocessor.adaptive_preprocessing_pipeline(
    image_path='noisy_document.jpg'
)

print("\n📊 이미지 품질 지표:")
for key, value in quality_metrics.items():
    print(f"  {key}: {value}")

# 전처리된 이미지 저장
cv2.imwrite('processed_document.jpg', processed_img)
cv2.imwrite('binary_document.jpg', binary_img)


In [ ]:
# 전처리 전후 OCR 비교
import pytesseract
import Levenshtein

ground_truth = "계약서 계약일자: 2025년 1월 15일 계약금액: 1,500,000원 담당자: 홍길동 (02-1234-5678)"

# OCR 실행
original = cv2.imread('noisy_document.jpg')
original_text = pytesseract.image_to_string(original, lang='kor+eng').strip()
processed_text = pytesseract.image_to_string(binary_img, lang='kor+eng').strip()

# 정확도 계산
def calc_accuracy(pred, truth):
    pred_clean = pred.replace(' ', '').replace('\n', '')
    truth_clean = truth.replace(' ', '').replace('\n', '')
    distance = Levenshtein.distance(pred_clean, truth_clean)
    return max(0, (1 - distance / max(len(pred_clean), len(truth_clean))) * 100)

original_acc = calc_accuracy(original_text, ground_truth)
processed_acc = calc_accuracy(processed_text, ground_truth)

print("\n" + "="*70)
print("📊 OCR 정확도 비교")
print("="*70)
print(f"\n노이즈 이미지 OCR 결과:")
print(f"  → {original_text}")
print(f"  정확도: {original_acc:.2f}%")
print(f"\n전처리 후 OCR 결과:")
print(f"  → {processed_text}")
print(f"  정확도: {processed_acc:.2f}%")
print("="*70)

improvement = processed_acc - original_acc
if improvement > 0:
    print(f"\n🎉 전처리로 +{improvement:.2f}%p 개선!")
else:
    print(f"\n💡 이 경우 전처리 효과: {improvement:.2f}%p")
    
# 시각화
fig, ax = plt.subplots(figsize=(10, 6))
engines = ['노이즈 이미지', '전처리 후']
accuracies = [original_acc, processed_acc]
colors = ['#e74c3c', '#2ecc71']

bars = ax.bar(engines, accuracies, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
ax.set_ylabel('정확도 (%)', fontproperties=fontprop, fontsize=12)
ax.set_title('전처리 전후 OCR 정확도 비교', fontsize=16, fontproperties=fontprop, pad=20)
ax.set_ylim(0, 100)
ax.grid(axis='y', alpha=0.3)

# 막대 위에 수치 표시
for i, (bar, acc) in enumerate(zip(bars, accuracies)):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 2,
            f'{acc:.1f}%',
            ha='center', va='bottom', fontsize=14, fontweight='bold', fontproperties=fontprop)

plt.tight_layout()
plt.show()

print("\n✅ 전처리 효과 분석 완료!")
